In [1]:
from team_game import TeamGame
import pandas as pd

games_details = pd.read_csv('games_details.csv')
games_grouped = games_details.groupby(['GAME_ID', 'TEAM_ID'])

game_to_stats = {}
for group_key, group_df in games_grouped:
    tg = TeamGame(group_df.iloc[0]['GAME_ID'], group_df.iloc[0]['TEAM_ID'])
    
    tg.fgm = group_df['FGM'].sum()
    tg.fga = group_df['FGA'].sum()
    tg.fg3m = group_df['FG3M'].sum()
    tg.ftm = group_df['FTM'].sum()
    tg.fta = group_df['FTA'].sum()
    tg.dreb = group_df['DREB'].sum()
    tg.oreb = group_df['OREB'].sum()
    tg.tov = group_df['TO'].sum()

    if tg.game_id in game_to_stats:
        game_to_stats[tg.game_id].append(tg)
    else:
        game_to_stats[tg.game_id] = [tg]

C:\Users\saraf\AppData\Local\Temp\ipykernel_26092\167922959.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details = pd.read_csv('games_details.csv')


In [2]:
import csv
from team_game import TeamGame

header = ['GAME_ID', 'DATE', 'SEASON', 'TEAM_ID', 'EFG', 'FTR', 'OREB_PCT', 'TOV_PCT', 'OFF_RTG', 'DEF_RTG', 'HOME', 'MOV']

games = pd.read_csv('games.csv')
with open('merged_games.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    for index, row in games.iterrows():
        if not(row['PTS_home'] > 0):
            continue
        g = game_to_stats[row['GAME_ID']]
        if len(g) != 2:
            continue
        t1, t2 = g[0], g[1]

        if t1.team_id == row['HOME_TEAM_ID']:
            writer.writerow([row['GAME_ID'], row['GAME_DATE_EST'], row['SEASON'], t1.team_id, t1.efg(), t1.ftr(), t1.oreb / (t1.oreb + t2.dreb), t1.tov_pct(), row['PTS_home'] / t1.poss(), row['PTS_away'] / t2.poss(), 1, row['PTS_home'] - row['PTS_away']])
            writer.writerow([row['GAME_ID'], row['GAME_DATE_EST'], row['SEASON'], t2.team_id, t2.efg(), t2.ftr(), t2.oreb / (t2.oreb + t1.dreb), t2.tov_pct(), row['PTS_away'] / t2.poss(), row['PTS_home'] / t1.poss(), 0, row['PTS_away'] - row['PTS_home']])
        else:
            writer.writerow([row['GAME_ID'], row['GAME_DATE_EST'], row['SEASON'], t2.team_id, t2.efg(), t2.ftr(), t2.oreb / (t2.oreb + t1.dreb), t2.tov_pct(), row['PTS_home'] / t2.poss(), row['PTS_away'] / t1.poss(), 0, row['PTS_home'] - row['PTS_away']])
            writer.writerow([row['GAME_ID'], row['GAME_DATE_EST'], row['SEASON'], t1.team_id, t1.efg(), t1.ftr(), t1.oreb / (t1.oreb + t2.dreb), t1.tov_pct(), row['PTS_away'] / t1.poss(), row['PTS_home'] / t2.poss(), 1, row['PTS_away'] - row['PTS_home']])

print("CSV file written successfully!")

CSV file written successfully!
